# Westeros Tutorial - Introducing the use of Excel files: Building a scenario by importing data from multiple ".xlsx" file

## Scope of this tutorial

This tutorial builds on "Westeros Tutorial Part 3.2.1", which explained how data from the Westeros Baseline scenario can be exported to an Excel (.xlsx) file using the function [`ixmp.Scenario.to_excel()`](https://docs.messageix.org/projects/ixmp/en/latest/api.html#ixmp.Scenario.to_excel) and how this data can be used to build a new scenario via [`ixmp.Scenario.read_excel()`](https://docs.messageix.org/projects/ixmp/en/latest/api.html#ixmp.Scenario.read_excel). This tutorial will look at the workflow of building a new scenario from data stored in an Excel file more carefully. 

**Pre-requisites**
- You have the *MESSAGEix* framework installed and working
- You have run Westeros baseline scenario (``westeros_baseline.ipynb``) and solved it successfully
- You have gone through Westeros Tutorial 4a, part 1 on importing data from an Excel file  (``westeros_baseline_using_xlsx_import_part1.ipynb``)

To do this, we are again going to build the Westeros Baseline, but this time combining elements from the original baseline tutorial with the import of data from multiple Excel files (see figure below - the red line indicates at which point of the process this tutorial will commence i.e. the source Excel files have already been generated). 

<img src='_static/westeros_baseline_xlsx_workflow_part2.jpg'>

The first part will create a scenario and basic parameters will be added including the modeling time-horizon and the discount rate. This code is a repetition of that used in the tutorial `westeros_baseline.ipynb`, which will be referred to as the "original" scenario throughout the remainder of this tutorial.

The second part will then import various parameters from Excel files, adding technologies, demand, and constraints to the model. We have chosen to distribute the import of data across various Excel files. This helps to outline a possible approach on how to structure the import methods if used for loading large amounts of data into scenarios. It further demonstrates the required data contents of the Excel files.

<div class="alert alert-block alert-success">
 
**Please Note:**

This tutorial specifically targets users looking to build larger models using the Excel import functionalities. The advantage of using multiple files, especially for large data sets, is that this provides a more transparent structuring of the underlying workflow. Scripts related to creating the Excel source files and the respective imports will be limited to certain model aspects. 
    
<u>An example:</u>
    
As shown in the figure above (right panel), a script can generate and import "technology" data, another the "demand" and a third generates "constraints". If alternative demand projects are to be used, for example to account for more factors influencing near term economic growth, then only a single script will need to be revised.  
    
This means that maintenance and updates can be performed more readily in the future. While acknowledging the fact that in some cases, for example when transitioning from other model platforms, importing data as demonstrated in this tutorial is possible, it is only advised to rely on these methods temporarily. Using multiple Excel files can aid in this respect. It allows the modeler to subsequently replace the use of `read_excel()` functionality in a more structured manner, updating the workflow for smaller more manageable scenario aspects.

## Step 1. Setting up the scenario

As in the "original" tutorial, we start by connecting to a database via `ixmp.Platform()`.

In [ ]:
import ixmp

# Loading Modeling platform
mp = ixmp.Platform()

In [ ]:
import message_ix

# Creating a new, empty scenario
scenario = message_ix.Scenario(
    mp, model="Westeros Electrified", scenario="baseline_xlsx", version="new"
)

## Model Structure

As in the original tutorial, basic characteristics of the model, including model years, location, and the energy system structure are defined. We also define the interest rate.

In [ ]:
# Defining historical and model (optimization) years
history = [690]
model_horizon = [700, 710, 720]
scenario.add_horizon(year=history + model_horizon, firstmodelyear=model_horizon[0])

In [ ]:
# Defining a spatial level and adding a node to it
node = "Westeros"
scenario.add_spatial_sets({"country": node})

Contrary to the original tutorial for building the Westeros model, we only define commodities and levels here. If additional `"commodity"` or `"level"` types are required later on, these can also be defined with uploading technologoy data using the Excel files. Either method is fine.
Note that we populated the set `"technology"` and `"mode"` in the original tutorial. We will address these in the second half of this tutorial.

In [ ]:
# Adding elements to MESSAGEix sets
scenario.add_set("commodity", ["electricity", "light"])
scenario.add_set("level", ["secondary", "final", "useful"])

In [ ]:
# Adding interest rate of 5% per model year
scenario.add_par("interestrate", model_horizon, value=0.05, unit="-")

Note that the data has not yet been commited. This is because at a minimum, the set technology needs to be defined, otherwise an error message will be raised when commiting the scenario. Hence, we will continue in this tutorial by adding data, step-by-step, in the process of which commits are automatically made.

## Step 2: Importing data from Excel


### 2.1 Supply and Demand (or Balancing Commodities)

Instead of using the `message_ix.Scenario.add_par()` for adding data to a MESSAGEix parameter, we import data from an xlsx file. The argument `add_units` has been set to `True`, so that any units which have not yet been specified in the modeling platform will be defined automatically.

In [ ]:
scenario.read_excel("westeros_baseline_demand.xlsx", add_units=True, commit_steps=False)

Open the above indicated xlsx ("westeros_baseline_demand.xlsx") file and browse its contents. You will find that it includes two sheets, of which we will need only one here (the other will be explained below). Namely, the sheet "demand", which contains the data based on the index structure of the parameter `"demand"`.

In [ ]:
print(scenario.idx_names("demand"))

In addition, numerical values have been added under the index-name `value` and the units have been specified under the index-name `unit`. These two index names are used for adding numerical values and units across all parameters.

### 2.2 Adding technologies

The next step is to add a basic configuration for technologies, using the data input file "westeros_baseline_technology_basic.xlsx".
This Excel file has the following sheets:

- mode
- technology
- capacity_factor
- input
- output
- technical_lifetime
- ix_type_mapping

The `ixmp` documentation ([“Scenario/model data”](https://docs.messageix.org/projects/ixmp/en/latest/file-io.html#excel-data-format) section of the “File formats and input/output” page) gives a complete description of the file format. To summarize:
- Most sheets have a name like "technology" or "input" that corresponds to a set or parameter in the MESSAGE formulation
- The "ix_type_mapping" sheet is used by the export/import code to keep track of this correspondence

The importer handles sets first, and then parameters. The sheets for sets *must* include all elements used by the parameter data; otherwise the data is invalid and a Python exception will be raised. For example, if data in the sheet `"capacity_factor"` refers to a technology which is not listed in the sheet "technology", the data are invalid. If the files are created by hand (instead of using the `to_excel()` method), it is important to ensure they are valid.

In [ ]:
scenario.read_excel(
    "westeros_baseline_technology_basic.xlsx", add_units=True, commit_steps=False
)

In order to view which technologies we have added, we can do the following:

In [ ]:
scenario.set("technology")

We can also look at the specific parameters by:

In [ ]:
scenario.par("capacity_factor")

### 2.3 Technological Diffusion and Contraction
As we have already undertaken an initial definition of technologies, and will now proceed to add additional parameters for already defined technologies, the input data file ("westeros_baseline_technology_constraint.xlsx") does not contain any sheets relating to sets. Still, we import the parameter `"growth_activity_up"` from there.

In [ ]:
scenario.read_excel(
    "westeros_baseline_technology_constraint.xlsx", add_units=True, commit_steps=False
)

### 2.4 Defining an Energy Mix (Model Calibration)
In this part, we import data of the historical years.

In [ ]:
scenario.read_excel(
    "westeros_baseline_technology_historic.xlsx", add_units=True, commit_steps=False
)

### 2.5 Investment, Fixed O&M and Variable O&M Costs
Here, we import the cost data from a separate file.

In [ ]:
scenario.read_excel(
    "westeros_baseline_technology_economic.xlsx", add_units=True, commit_steps=False
)

## Time to Solve the Model

In [ ]:
scenario.set_as_default()

In [ ]:
scenario.solve()

## Check the objective value
Finally, we can check the objective value of the original baseline scenario with that of the newly created version.

In [ ]:
# Objective value of the original 'baseline' scenario.
base = message_ix.Scenario(mp, model="Westeros Electrified", scenario="baseline")
base.var("OBJ")["lvl"]

In [ ]:
# Objective value of scenario built using the Excel file.
scenario.var("OBJ")["lvl"]

In [ ]:
mp.close_db()